# Download genomes
Examining the study by {cite:t}`Ide2022` made me realize I can tap into assembled genomes on Genbank, thereby expanding the 16S dataset.
Genomes of Endozoicomonas were downloaded from Genbank in this script. 16S sequences will be ripped from this genomes, and put into their own fasta file for later analysis.

## Setup environment

In [2]:
from Bio import Entrez, SeqIO
from Bio.SeqRecord import SeqRecord
import os
import logging
from Bio.Entrez.Parser import DictionaryElement
from pandas import DataFrame
from Bio.Entrez import HTTPError
from Bio.Entrez.Parser import ListElement
from pandas import DataFrame
import urllib.parse
import ncbi_genome_download as ngd

logging.basicConfig(level=logging.INFO)

In [3]:
Entrez.email = "kaedeito@student.ubc.ca"
# This line sets the name of the tool that is making the queries
Entrez.tool = "download_16s.ipynb"

logger = logging.getLogger("download_genome")

In [4]:
dir_path = os.path.realpath("..\\..\\datasets\\full_analysis")
os.makedirs(dir_path, exist_ok=True)

## Define search functions and their helpers

The genome and assembly (rightfully so) restricts the number of queries to 10, even if we set the retmax. Therefore, we need some helper functions to create (start, end) tuples for the queries.

In [5]:
def generate_ranges(max)-> list[tuple[int, int]]:
  """
  Generate ranges for the entrez query pagination.

  Args:
    `max`: the maximum number of records to be returned
  """
  q, rem = divmod(max, 10)
  pagination_list: list[tuple[int, int]] = []

  q_range = range(q + 1)
  for i in q_range:
    start = i * 10
    if start > 0:
      start += 1
    if start > max:
      continue
    next = i + 1
    if next in q_range:
      pagination_list.append((start, next * 10))
    else:
      pagination_list.append((start, (i * 10) + rem))
  return pagination_list

Let's test that it works the way we think it does.

In [6]:
print(generate_ranges(9))
print(generate_ranges(10))
print(generate_ranges(20))
print(generate_ranges(21))
print(generate_ranges(22))
print(generate_ranges(29))

[(0, 9)]
[(0, 10)]
[(0, 10), (11, 20)]
[(0, 10), (11, 20), (21, 21)]
[(0, 10), (11, 20), (21, 22)]
[(0, 10), (11, 20), (21, 29)]


This function will search by using normal terms, and then by using the taxanomic id.

In [7]:
def search_genome(max: int) -> tuple[int, DataFrame]:
  """
    Search the genome database for the term "Endozoicomonas" and return the number of records and the dataframe of the results' metadata.
  """
  term = "Endozoicomonas"

  count = 0
  list_accession: list[dict[str, str | None]] = []

  try:
    pagination_list: list[tuple[int, int]] = generate_ranges(max)

    logger.debug(pagination_list)

    for start, end in pagination_list:
      handle = Entrez.esearch(db="genome", term=term, retstart=start, retmax=end)
      res = Entrez.read(handle)
      handle.close()

      if isinstance(res, DictionaryElement):
        if len(res["IdList"]) == 0:
          break

        assembly_ID_objects=Entrez.read(Entrez.efetch(db="Genome",id=res["IdList"],rettype="docsum", retmode="xml"))

        if isinstance(assembly_ID_objects, ListElement):
          logger.debug(type(assembly_ID_objects[0]))
          count += len(assembly_ID_objects)
          logger.debug(f"count: {count}")
          list_accession.extend([{
            "taxon_id": assembly_ID_object["Id"],
            "organism_name": assembly_ID_object["Organism_Name"],
            "assembly_accession": assembly_ID_object["Assembly_Accession"] if assembly_ID_object["Assembly_Accession"] != "" else None
            } for assembly_ID_object in assembly_ID_objects])

    # Replace empty strings with None
    list_accession_filtered: list[dict[str, str]] = [accession for accession in list_accession if accession["assembly_accession"] is not None] # type: ignore

    # For some reason, the package cannot find the GCA accession, so we have to replace it with GCF.
    # https://www.ncbi.nlm.nih.gov/books/NBK50679/#RefSeqFAQ.what_is_the_difference_between_1
    for accession in list_accession_filtered:
      accession["assembly_accession"] = accession["assembly_accession"].replace("GCA", "GCF")

    # Convert to dataframe, tabular format
    df = DataFrame(list_accession_filtered)
    return count, df
  except HTTPError as err:
    logger.error(err.read())
    return 0, DataFrame()
  except Exception as err:
    logger.exception(err)
    return 0, DataFrame()

## Define download functions and their helpers

[`BioPython`](https://biopython.org/wiki/Documentation) {cite}`Cock_2009` does not have in-built functionality to download NCBI genome assemblies. Therefore, we will use `ncbi_genome_download` {cite}`blin_2023_8192486` pip package to download the files.

In [8]:
def download_genome(max = 20):
  """
    Download the genome data for the term "Endozoicomonas" and save it to the `../../datasets/full_analysis` directory.
  """
  count, df = search_genome(max)
  assembly_ids = df['assembly_accession'].to_list()
  try:
    # https://github.com/kblin/ncbi-genome-download/#citing-ncbi-genome-download
    ngd.download(assembly_accessions=assembly_ids, output=dir_path, file_formats=['genbank', 'gff'])
  except HTTPError as err:
    logger.error(urllib.parse.unquote(err.read()))
  except Exception as err:
    logger.exception(err)

In [9]:
import gzip
import shutil

def unpack_gz():
  """
    Unpack the `.gz` files in the `../../datasets/full_analysis/refseq/bacteria` directory.
    If files are already unpacked, add them to the list of unpacked files.
    This is necessary because the `ncbi-genome-download` package does not unpack the files.
    The files are unpacked in the same directory as the `.gz` files.

    Returns:
      `list_unpacked_gbff`: a list of the paths to the unpacked `.gbff` files

      `list_unpacked_gff`: a list of the paths to the unpacked `.gff` files
  """
  list_unpacked_gbff: list[str] = []
  list_unpacked_gff: list[str] = []
  parent_dir = os.path.join(dir_path, "refseq", "bacteria")

  # iterate through genome accession id directories
  for dir in os.listdir(parent_dir):
    # check if directory
    if os.path.isdir(os.path.join(parent_dir, dir)):
      # list files within directory
      file_list = os.listdir(os.path.join(parent_dir, dir))
      for file_path in file_list:
        # if files are already unpacked, add to list
        if len(file_list) > 3:
          full_file_path = os.path.join(parent_dir, dir, file_path)
          if file_path.endswith("gff"):
            list_unpacked_gff.append(full_file_path)
          elif file_path.endswith("gbff"):
            list_unpacked_gbff.append(full_file_path)
        # Otherwise unpack the files
        else:
          if file_path.endswith(".gz"):
            gz_file_name = os.path.join(parent_dir, dir, file_path)
            logger.debug(gz_file_name)
            with gzip.open(gz_file_name, 'rb') as f_in:
              file_name = os.path.join(parent_dir, dir, file_path.replace(".gz", ""))

              if file_name.endswith(".gbff"):
                list_unpacked_gbff.append(file_name)
              elif file_name.endswith(".gff"):
                list_unpacked_gff.append(file_name)
              logger.debug(file_name)
              with open(file_name, 'wb') as f_out:
                  shutil.copyfileobj(f_in, f_out)

  return list_unpacked_gbff, list_unpacked_gff


## Download genomes (gz)

In [10]:
list_in_dir = os.listdir(os.path.join(dir_path, "refseq", "bacteria"))

# if there are no folders already in the directory, download
if len(list_in_dir) == 0:
  download_genome(10)

unpacked_gbff, unpacked_gff = unpack_gz()

## Load genomes into memory

In [11]:
# https://biopython.org/wiki/GFF_Parsing
def load_genbank(list_gbff: list[str]) ->list[SeqRecord]:
  """
    Load the genbank files into a list of SeqRecords.
  """
  results: list[SeqRecord] = []
  for file_path in list_gbff:
    if not file_path.endswith(".gbff"):
      raise Exception("File is not a .gbff file")
    logger.debug(file_path)
    file = open(file_path, "r")
    record = list(SeqIO.parse(file, "genbank"))
    results.extend(record)
  return results

records = load_genbank(unpacked_gbff)

In [12]:
first_rec = records[0]
print(first_rec.features)

[SeqFeature(SimpleLocation(ExactPosition(0), ExactPosition(5569560), strand=1), type='source', qualifiers=...), SeqFeature(SimpleLocation(BeforePosition(0), ExactPosition(5834), strand=-1), type='gene', qualifiers=...), SeqFeature(SimpleLocation(BeforePosition(0), ExactPosition(5834), strand=-1), type='CDS', qualifiers=...), SeqFeature(SimpleLocation(ExactPosition(6104), ExactPosition(7220), strand=1), type='gene', qualifiers=...), SeqFeature(SimpleLocation(ExactPosition(6104), ExactPosition(7220), strand=1), type='CDS', qualifiers=...), SeqFeature(SimpleLocation(ExactPosition(7340), ExactPosition(7416), strand=-1), type='gene', qualifiers=...), SeqFeature(SimpleLocation(ExactPosition(7340), ExactPosition(7416), strand=-1), type='tRNA', qualifiers=...), SeqFeature(SimpleLocation(ExactPosition(7510), ExactPosition(9169), strand=-1), type='gene', qualifiers=...), SeqFeature(SimpleLocation(ExactPosition(7510), ExactPosition(9169), strand=-1), type='CDS', qualifiers=...), SeqFeature(Simple

In [13]:
from Bio.SeqFeature import SeqFeature

def find_16S_gbff(records: list[SeqRecord]):
  """
    Find the 16S rRNA genes in the dict of SeqRecords.

    Args:
      `records`: the list of SeqRecords to search

    Returns:
      `res`: the dict of { [record.id]: SeqRecord } that contain 16S rRNA genes.
  """
  res: dict[str, SeqFeature] = {}
  for record in records:
    feat_list: list[SeqFeature] = record.features
    for feature in feat_list:
      if feature.type == "rRNA":
        if "product" in feature.qualifiers:
          if "16S ribosomal RNA" in feature.qualifiers["product"][0]:
            res[record.id] = feature
  return res

res = find_16S_gbff(records)
print(res)

{'NZ_JOJP01000001.1': SeqFeature(SimpleLocation(ExactPosition(5030037), ExactPosition(5031580), strand=-1), type='rRNA', qualifiers=...), 'NZ_JOKH01000001.1': SeqFeature(SimpleLocation(ExactPosition(1003134), ExactPosition(1004675), strand=-1), type='rRNA', qualifiers=...), 'NZ_JOKH01000002.1': SeqFeature(SimpleLocation(ExactPosition(686460), ExactPosition(688001), strand=-1), type='rRNA', qualifiers=...), 'NZ_JOKH01000013.1': SeqFeature(SimpleLocation(BeforePosition(0), ExactPosition(942), strand=-1), type='rRNA', qualifiers=...), 'NZ_LASA01000010.1': SeqFeature(SimpleLocation(ExactPosition(26523), AfterPosition(26885), strand=1), type='rRNA', qualifiers=...), 'NZ_LASA01000015.1': SeqFeature(SimpleLocation(BeforePosition(0), ExactPosition(520), strand=-1), type='rRNA', qualifiers=...), 'NZ_LASA01000025.1': SeqFeature(SimpleLocation(BeforePosition(0), ExactPosition(654), strand=-1), type='rRNA', qualifiers=...), 'NZ_LASA01000026.1': SeqFeature(SimpleLocation(BeforePosition(0), ExactPos